<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/ann-for-survey-sampling/blob/main/notebooks/ann_paper_sim_study_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libomp-dev
!pip install faiss-gpu

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
import faiss
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean
from sklearn.neighbors import NearestNeighbors
res = faiss.StandardGpuResources() 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [51]:
def kdtree_impute(tree, sample, data, y, x, eps = 0):
  nns = tree.query(sample[:,x], k = 1, eps = eps)
  res = np.mean(data[nns[1]][:, ys], axis = 0)
  return res

## Simulation 3A

Simulation study taken from: Kim, J. K., & Wang, Z. (2018). Sampling Techniques for Big Data Analysis. International Statistical Review, 1, 1–15. https://doi.org/10.1111/insr.12290

In [ ]:
np.random.seed(123)
N = 1000000
x1 = np.random.normal(loc=1.0,scale=1.0,size=N)
x2 = np.random.exponential(scale=1.0, size = N)
epsilon = np.random.normal(size=N)

### target variables
y1 = 1 + x1 + x2 + epsilon
y2 = 0.5*(x1 - 1.5)**2 + x2 + epsilon
## propensity scores
p1 = np.exp(x2) / (1 + np.exp(x2))
p2 = np.exp(-0.5 + 0.5*(x2-2)**2) / (1 + np.exp(-0.5 + 0.5*(x2-2)**2))

data = np.column_stack((x1,x2,y1,y2,p1,p2)).astype('float32')
data[:3]

array([[-0.0856306 ,  0.5546646 ,  0.90054107,  1.2432839 ,  0.63521713,
         0.632858  ],
       [ 1.9973454 ,  1.1975824 ,  4.455222  ,  1.5815529 ,  0.7680944 ,
         0.4556015 ],
       [ 1.2829785 ,  0.84342945,  3.0256119 ,  0.76618266,  0.699187  ,
         0.542107  ]], dtype=float32)

In [ ]:
R = 10
xs = [0,1]
ys = [2,3]
ps = [4,5]

results_faiss_500 = np.zeros(shape = (R, 2))
results_faiss_1000 = np.zeros(shape = (R, 2))
results_kdtree_500 = np.zeros(shape = (R, 2))
results_kdtree_1000 = np.zeros(shape = (R, 2))

for r in range(R):
  print(r)
  np.random.seed(r)
  ## big data sample
  big_p1 = np.random.binomial(n=1, p = p1, size = N)
  big_p2 = np.random.binomial(n=1, p = p2, size = N)    
  ## random samples
  s500 = np.random.choice(a = data.shape[0], size = 500, replace = False)
  s1000 = np.random.choice(a = data.shape[0], size = 1000, replace = False)
  ## kdtree (exact)
  kdtree = KDTree(data[big_p2==1][:, xs], leafsize = 100)
  results_kdtree_500[r, :] = kdtree_impute(kdtree, data[s500], data[big_p2==1], ys, xs, eps = 100)
  results_kdtree_1000[r, :] = kdtree_impute(kdtree, data[s1000], data[big_p2==1], ys, xs, eps = 100)
  ## faiss
  big_data = data[big_p2==1][:, xs].copy()
  sam_data_500 = data[s500][:, xs].copy()
  sam_data_1000 = data[s1000][:, xs].copy()
  index_flat = faiss.IndexFlatL2(len(xs))
  gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
  gpu_index_flat.add(big_data)
  D_500, I_500 = gpu_index_flat.search(sam_data_500, k = 1) 
  D_1000, I_1000 = gpu_index_flat.search(sam_data_1000, k = 1) 
  ind_500 = [i[0] for i in I_500]
  ind_1000 = [i[0] for i in I_1000]
  results_faiss_500[r,:]=np.mean(data[big_p2==1][ind_500][:, ys], axis=0)
  results_faiss_1000[r,:]=np.mean(data[big_p2==1][ind_1000][:, ys], axis=0)

0
1
2
3
4
5
6
7
8
9


In [ ]:
expected = np.stack(
    [np.mean(results_kdtree_500, axis=0),
     np.mean(results_faiss_500, axis=0),
     np.mean(results_kdtree_1000, axis=0),
     np.mean(results_faiss_1000, axis=0)
     ]
) 

stderrs =  np.stack(
    [np.std(results_kdtree_500, axis=0),
     np.std(results_faiss_500, axis=0),
     np.std(results_kdtree_1000, axis=0),
     np.std(results_faiss_1000, axis=0)
     ]
)

bias = expected - np.mean(data[:,ys], axis=0)
mse = bias**2 + stderrs**2

print("===== bias =====")
print(bias)
print("===== se =====")
print(stderrs)
print("===== mse =====")
print(mse)

===== bias =====
[[-0.00685911 -0.02907437]
 [-0.00759137 -0.02993623]
 [ 0.0285023   0.01061232]
 [ 0.03065183  0.01280404]]
===== se =====
[[0.05752631 0.10151303]
 [0.05390689 0.0999129 ]
 [0.04877032 0.03980684]
 [0.05182396 0.04317026]]
===== mse =====
[[0.00335632 0.01115022]
 [0.00296358 0.01087877]
 [0.00319093 0.00169721]
 [0.00362526 0.00202762]]


In [ ]:
## save results
results = {
    "results_kdtree_500" : pd.DataFrame(results_kdtree_500),
    "results_faiss_500" : pd.DataFrame(results_faiss_500),
    "results_kdtree_1000": pd.DataFrame(results_kdtree_1000),
    "results_faiss_1000": pd.DataFrame(results_faiss_1000),
           }

f = open("kdtree_faiss_500_1000k.pkl","wb")
pickle.dump(results,f)
f.close()

## Simulation 3B

Modified case from: Yang, S., & Kim, J. K. (2019). Nearest neighbour imputation for general parameter estimation in survey sampling. In The Econometrics of Complex Survey Data: Theory and Applications (Vol. 39, pp. 209–234)

In [2]:
def kdtree_impute2(tree, data_obs, data_not, y, x, N, eps = 0):
  nns = tree.query(data_not[:,xs], k = 1, eps = eps)
  res = (np.sum(data_obs[nns[1]][:, ys], axis = 0) + np.sum(data_obs[:, ys], axis = 0)) / N
  return res

In [15]:
np.random.seed(123)
N = 50000
x1 = np.random.uniform(size = N)
x2 = np.random.uniform(size = N)
x3 = np.random.uniform(size = N)
x4 = np.random.normal(size = N)
x5 = np.random.normal(size = N)
x6 = np.random.normal(size = N)
epsilon = np.random.normal(size=N)

### target variables
y1 = -1 + x1 + x2 + epsilon
y2 = -1.5 + x1 + x2 + x3 + x4 + epsilon
y3 = -1.5 + x1 + x2 + x3 + x4 + x5 + x6 + epsilon
y4 = -1 + x1 + x2 + x1**2 + x2**2 -2/3 + epsilon
y5 = -1 + x1 + x2 + x3 + x4 + x1**2 + x2**2 -2/3 + epsilon
y6 = -1 + x1 + x2 + x3 + x4 + x5 + x6 + x1**2 + x2**2 -2/3 + epsilon

## propensity scores
p1 = np.exp(1 + x1 + x2 + x3 + x4 + x5 + x6) / (1 + np.exp(1 + x1 + x2 + x3 + x4 + x5 + x6))
p2 = np.exp(1 + x1 + x2) / (1 + np.exp(1 + x1 + x2))

data = np.column_stack((x1,x2,x3,x4,x5,x6,y1,y2,y3,y4,y5,y6)).astype('float32')
data[:3]

array([[ 0.6964692 ,  0.36086547,  0.20932843,  0.47665665,  0.80429375,
        -0.9894137 ,  0.42543107,  0.61141616,  0.42629617,  0.3740576 ,
         1.0600427 ,  0.87492275],
       [ 0.28613934,  0.22535679,  0.2937351 ,  0.6701647 ,  1.2931948 ,
         1.033963  ,  0.25331086,  0.71721065,  3.0443685 , -0.2806944 ,
         0.6832054 ,  3.010363  ],
       [ 0.22685145,  0.50813043,  0.05571789,  0.5033514 ,  1.5591047 ,
         0.13504769,  0.36729714,  0.42636642,  2.1205187 ,  0.0102886 ,
         0.5693579 ,  2.2635102 ]], dtype=float32)

In [ ]:
R = 500
xs = [0,1,2,3,4,5]
ys = [6,7,8,9,10,11]

results_faiss = np.zeros(shape = (R, 6))
results_kdtree = np.zeros(shape = (R, 6))
results_pmm = np.zeros(shape = (R, 6))
results_identical = np.zeros(shape = (R,1))

for r in range(R):
  print(r)
  np.random.seed(r)
  ## big data sample
  big_p1 = np.random.binomial(n=1, p = p1, size = N)
  ## data
  data_obs = data[big_p1 == 1].copy()
  data_not = data[big_p1 != 1].copy()
  ## running
  X_obs = data_obs[:, xs]
  X_obs = sm.add_constant(X_obs)
  y_obs = data_obs[:, ys[5]]
  ols_y = sm.OLS(y_obs, X_obs).fit()
  X_not = data_not[:, xs]
  X_not = sm.add_constant(X_not)
  y_not = np.dot(X_not, ols_y.params)
  ## nearest nn
  neigh = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(ols_y.fittedvalues.reshape(-1, 1))
  inds = neigh.kneighbors(y_not.reshape(-1,1), return_distance=False)
  results_pmm[r, 5] = (np.sum(y_obs) + np.sum(y_obs[inds]))/N
  ## faiss
  data_obs_faiss = ols_y.fittedvalues.copy().reshape(-1, 1).astype('float32')
  data_not_faiss = y_not.copy().reshape(-1, 1).astype('float32')
  index_flat = faiss.IndexFlatL2(1)
  gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
  gpu_index_flat.add(data_obs_faiss)
  faiss_dists, faiss_inds = gpu_index_flat.search(data_not_faiss, k = 1) 
  results_faiss[r, 5] = (np.sum(y_obs) + np.sum(y_obs[faiss_inds]))/N
  ## identical
  results_identical[r, 0] = sum(inds.flatten() == faiss_inds.flatten()) / len(data_not_faiss)
  ## kdtree
  #kdtree = KDTree(data_obs_faiss, leafsize = 10)
  #dd, ii = kdtree.query(data_not_faiss, k=1)
  #results_kdtree[r,:]= (np.sum(data_obs[ii][:, ys], axis=0) + np.sum(data_obs[:, ys], axis=0)) / N

In [21]:
expected = np.stack(
    [np.mean(results_kdtree, axis=0),
     np.mean(results_faiss, axis=0)
     ]
) 

stderrs =  np.stack(
    [np.std(results_kdtree, axis=0),
     np.std(results_faiss, axis=0)
     ]
)

bias = expected - np.mean(data[:,ys], axis=0)
mse = bias**2 + stderrs**2

print("===== bias =====")
print(bias[:,5])
print("===== se =====")
print(stderrs[:, 5])
print("===== mse =====")
print(np.round(mse,4))
print(np.round(expected,4))
print("===== pmm =====")
print(np.mean(results_pmm[:, 5])-np.mean(data[:,ys[5]], axis=0))
print(np.std(results_pmm[:, 5]))

===== bias =====
[-0.51053309  0.00152759]
===== se =====
[0.        0.0037306]
===== mse =====
[[0.000e+00 1.000e-04 1.000e-04 0.000e+00 2.577e-01 2.606e-01]
 [0.000e+00 1.000e-04 1.000e-04 0.000e+00 2.577e-01 0.000e+00]]
[[0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.5121]]
===== pmm =====
0.0014384213751220676
0.003692713071889185


0.6216522566575691

## Simulation 3C

Study from Yang, S., & Kim, J. K. (2020). Doubly robust inference when combining probability and non-probability samples with high dimensional. Journal of the Royal Statistical Society. Series B: Statistical Methodology, 82(2), 445–465. https://doi.org/10.1111/rssb.12354

R code (source: https://github.com/shuyang1987/IntegrativeFPM)

```r
set.seed(1234)
## population size
N <- 10000
## x is a p-dimensional covariate
p <- 50
x <- matrix( rnorm(N*p,0,1),N,p)
## y is a continuous outcome 
beta0 <- c(1,1,1,1,1,rep(0,p-4))
y <- cbind(1,x)%*%beta0 + rnorm(N,0,1)
true <- mean(y)
## y2 is a binary outcome
ly2 <- (cbind(1,x)%*%beta0)
ply <- exp(ly2)/(1+exp(ly2))
y2 <- rbinom(N,1,ply)
true2 <- mean(y2)
## A.set is a prob sample: SRS
## sampling probability into A is known when estimation
nAexp <- 1000
probA <- rep(nAexp/N,N)
A.index <- rbinom(N,size = 1,prob = probA)
A.loc <- which(A.index == 1)
nA <- sum(A.index == 1)
sw.A <- 1/probA[A.loc]
x.A <- x[A.loc,]
y.A <- rep(NA,nA) # y is not observed in Sample A
y2.A <- rep(NA,nA)
## B.set is a nonprob sample
## sampling probability into B is unknown when estimation
nBexp <- 2000
alpha0 <- c(-2,1,1,1,1,rep(0,p-4))
probB <- (1+exp(-cbind(1,x)%*%alpha0))^(-1) 
B.index <- rbinom(N,size = 1,prob = probB)
B.loc <- which(B.index == 1)
nB <- sum(B.index)
x.B <- x[B.loc,]
y.B <- y[B.loc]
y2.B <- y2[B.loc]
## combined dataset
y.AB <- c(y.A,y.B)
y2.AB <- c(y2.A,y2.B)
x.AB <- rbind(x.A,x.B)
deltaB <- c(rep(0,nA),rep(1,nB))
sw <- c(sw.A,rep(1,nB))
```